In [1]:
import os
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time

import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

df_records = pd.read_csv('Category_Tracking.csv')
print(df_records)

                        Category     Abbrv  Amount of URLS  \
0  Business Support and Supplies       bss           24473   
1            Cars and Automotive      cars           11698   
2   Construction and Engineering        CE           24827   
3       Computer and Electronics  computer            9187   
4        Fashion and Accessories       F&A            7020   
5      Healthcare and Medication       H&M           13836   
6               Home and Living        H&L           16913   
7                 Retail Stores     retail           12001   
8             Sports and Hobbies    sports            1655   
9      Trading and Manufacturing   trading           12001   

   URLS last search index  Amount of Data  Data last search index  \
0                   24473           21116                     NaN   
1                    3160            3000                     NaN   
2                   24827           24821                 24821.0   
3                    4502            4500

In [2]:

df_records = pd.read_csv('Category_Tracking.csv')
url_csv_filename = lambda abbrv: "yellowpages_urls_" + abbrv + ".csv"
data_csv_filename = lambda abbrv: "yellowpages_data_" + abbrv + ".csv"
cleaned_csv_filename = lambda abbrv: "cleaned_" + abbrv + ".csv"

tasks = [
    {'category':1, 'num':9000},
]

def scrape_data(category,num):
    start_time = time.time()
    category_data = df_records.loc[category]
    start_index = category_data["URLS last search index"] +1
    end_index = start_index + num if start_index + num < category_data["Amount of URLS"] else category_data["Amount of URLS"]
    print(end_index)
    df = pd.read_csv(url_csv_filename(category_data['Abbrv']))
    companies_data = []
    
    
    # Iterate over the specified range of indices in the DataFrame
    for index in range(start_index, end_index - 2):
        url = df.loc[index, 'Company Profile URL']

        # Define headers to mimic a browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.6099.110 Safari/537.36',
            'Referer': 'https://www.yellowpages.my/services/l/business-support-supplies'
        }

        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.ok:
            # Create a BeautifulSoup object and specify the parser
            soup = BeautifulSoup(response.text, 'html.parser')

            # Define CSS selectors
            title_selector = 'div.title.yp-h2'
            telephone_selector = 'body > app-root > div > app-root > app-profile > div > div > div.right > div.info.ng-star-inserted > div.map > div > div.item.address-cont > div > div > div > div > a:nth-child(1)'
            email_selector = 'body > app-root > div > app-root > app-profile > div > div > div.right > div.info.ng-star-inserted > div.map > div > div.item.address-cont > div > div > div > div > a:nth-child(2)'

            # Extract data using selectors
            company_title = soup.select_one(title_selector).text.strip() if soup.select_one(title_selector) else 'Not found'
            telephone_number = soup.select_one(telephone_selector).text.strip() if soup.select_one(telephone_selector) else 'Not found'
            email_address = soup.select_one(email_selector).text.strip() if soup.select_one(email_selector) else 'Not found'

            # Append the data to the list
            companies_data.append({
                'Company Name': company_title,
                'Email': email_address,
                'Telephone': telephone_number
            })
            
            print(f"Current Saved Index {index}", end = "\r")
        else:
            print(f"\nFailed to retrieve webpage for index {index}.")

        # Add a delay to avoid being blocked or to mimic human interaction
        time.sleep(1)
        
    # Convert the list of dictionaries to a DataFrame
    new_companies_df = pd.DataFrame(companies_data)

    # End the timer
    end_time = time.time()
    total_time = end_time - start_time

    # Append this data to the existing CSV file
    new_companies_df.to_csv(data_csv_filename(category_data['Abbrv']), mode='a', header=False, index=False)

    print(f"Appended data for {len(companies_data)} companies to the CSV file.")
    print(f"Total time taken: {total_time} seconds.")
    df_records.loc[category,'URLS last search index'] = end_index
    df_records.loc[category,'Amount of Data'] = len(pd.read_csv(data_csv_filename(category_data['Abbrv'])))
    df_records.to_csv('Category_Tracking.csv', index=False)
    
    
    
for task in tasks:
    scrape_data(task['category'],task['num'])

df_records = pd.read_csv('Category_Tracking.csv')
print(df_records)

11698
Current Saved Index 4993
Failed to retrieve webpage for index 4994.
Current Saved Index 4997
Failed to retrieve webpage for index 4998.
Current Saved Index 5836
Failed to retrieve webpage for index 5837.

Failed to retrieve webpage for index 5838.
Current Saved Index 5840
Failed to retrieve webpage for index 5841.

Failed to retrieve webpage for index 5842.
Current Saved Index 5848
Failed to retrieve webpage for index 5849.

Failed to retrieve webpage for index 5850.
Current Saved Index 5851
Failed to retrieve webpage for index 5852.

Failed to retrieve webpage for index 5853.

Failed to retrieve webpage for index 5854.

Failed to retrieve webpage for index 5855.
Current Saved Index 5861
Failed to retrieve webpage for index 5862.
Current Saved Index 5863
Failed to retrieve webpage for index 5864.
Current Saved Index 5980
Failed to retrieve webpage for index 5981.

Failed to retrieve webpage for index 5982.
Current Saved Index 5983
Failed to retrieve webpage for index 5984.
Curren

In [14]:
row = df_records.loc[2]
print(row['Abbrv'])

CE
